In [ ]:
import networkx as nx
import heapq

In [ ]:
G = nx.DiGraph()
edges = [(1, 2, 10),
         (2, 3, 1),
         (3, 4, 10),
         (1, 8, 20),
         (8, 2, 1),
         (2, 6, 1),
         (6, 7, 1),
         (7, 8, 1),
         (2, 7, 3),
         (7, 5, 15),
         (3, 5, 18),
         (5, 4, 1)]

# A 1
# B 2
# C 3
# D 4
# E 5
# F 6
# G 7
# H 8
# I 9

G.add_weighted_edges_from(edges)

In [ ]:
nx.draw(G, with_labels=True)

In [ ]:
def reverse(graph):
  Gr = nx.DiGraph()
  # for node in graph.nodes():
  #   Gr.add_node(node)

  Gr.add_edges_from((v,u,d) for u,v,d in graph.edges(data=True))
  return Gr

In [ ]:
GR = reverse(G)

In [ ]:
nx.draw(GR, with_labels=True)

In [ ]:
class Path:
  def __init__(self):
    self.route = []
    self.edges = {}
    self.length = 0
    self.lb = 0
    self.cls = None
    self.isActive = True

  def __str__(self):
    return "route: " + str(self.route) + " edges: " + str(self.edges) + " length: " + str(self.length) + " lb: " + str(self.lb) + " cls: " + str(self.cls)
  
  def __lt__(self, other):
    return self.lb < other.lb #PQ kontrolü için


  def LB1(self, node, distances):
    return self.length + distances[node] 

  # result dict arrayi olarak tutuluyor
  def LB2(self, threshold, result):
    lb2 = 0

    for old_path in result: 

      old_path_edges = old_path.edges

      common_edges = set(old_path_edges.keys()).intersection(set(self.edges.keys()))

      intersection_length = sum(old_path_edges[e] for e in common_edges)

      current_lb2 = intersection_length * (1+1/threshold) - old_path.length
      lb2 = max(lb2, current_lb2)

    return lb2

  
  def Sim(self, threshold, result):
    flag = True

    for old_path in result: 
      old_path_edges = old_path.edges

      common_edges = set(old_path_edges.keys()).intersection(set(self.edges.keys()))
      intersection_length = sum(old_path_edges[e] for e in common_edges)

      # union_edges = set(new_path_edges.keys()).union(set(old_path_edges.keys()))
      # union_length = sum(new_path_edges[e] for e in union_edges if e in new_path_edges)
      # union_length += sum(old_path_edges[e] for e in union_edges if e in old_path_edges)
      # union_length -= intersection_length
      union_length = self.length + old_path.length - intersection_length

      similarity = intersection_length / union_length

      if similarity >= threshold:
        flag = False
        break  

    return flag
  
  def tail(self):
    return self.route[-1] if self.route else None

  def head(self):
    return self.route[0] if self.route else None

  def contains(self, vertex):
    return vertex in self.route


In [ ]:
def ConstructPartialSPT(graph, v):
  global distances, isSettled, PQ, parent

  if isSettled[v]:
    return distances[v]

  while PQ:
    cost, node = heapq.heappop(PQ)

    if cost > distances[node]:
      continue

    if not isSettled[node]:
      isSettled[node] = True

      for neighbor, data in graph[node].items():

        if not isSettled[neighbor]:        
          new_cost = cost + data['weight']

          if new_cost < distances[neighbor]:
            distances[neighbor] = new_cost
            parent[neighbor] = node
            heapq.heappush(PQ, (new_cost, neighbor))

      if node == v:
        return distances[v]      
      
  return float('inf')

In [ ]:
# path dönen dijkstra
def dijkstra(graph, src, dest):

  if src == dest:
    return {}, 0

  heap = [(0, src, [])]
  visited = set()

  while heap:
    cost, node, path = heapq.heappop(heap)

    if node in visited:
      continue
    visited.add(node)

    if node == dest:
      shortest_path = Path()

      for i in range(len(path)):
        if i < len(path) - 1:
          u, v = path[i], path[i+1]
          shortest_path.edges[(u,v)] = graph[u][v]['weight']
          shortest_path.length += graph[u][v]['weight']
          shortest_path.route.append(u)
        else:
          u, v = path[i], dest
          shortest_path.edges[(u,v)] = graph[u][v]['weight']
          shortest_path.length += graph[u][v]['weight']
          shortest_path.route.append(u)
          shortest_path.route.append(v)

      return shortest_path

    for neighbor, data, in graph[node].items():
      if neighbor not in visited:
        new_cost = cost + data['weight']
        heapq.heappush(heap, (new_cost, neighbor, path + [node]))

  return None

In [ ]:
distances = {}
isSettled = {}
PQ = []
parent = {}
result_set = []

for node in GR.nodes():
  distances[node] = float('inf')
  isSettled[node] = False
  parent[node] = None

dest = 4 # D vertexi destination noktamız
heapq.heappush(PQ, (0, dest))
distances[dest] = 0

In [ ]:
# p = Path()
# p.edges = {(1,2): 10,
#            (2,3): 1,
#            (3, 4): 10}
# result = []
# result.append(p)

first_shortest_path = dijkstra(G, 1, 4)
first_shortest_path.__dict__

In [ ]:
# path = Path()
# path.edges = {(1,2): 10,
#         (2,6): 1,
#         (6, 7): 1,
#         (7, 5): 15,
#         (5, 4): 1}

# LB2(path, 0.5)

In [ ]:
def FindKSPD(graph, src, dest, k, threshold):
  global result_set, distances

  shortest_path = dijkstra(graph, src, dest)
  result_set.append(shortest_path)

  global_PQ = []

  for vertex in shortest_path.route[:-1]:
    # print("----")
    # print(f"vertex: {vertex}")

    for neighbor in graph[vertex]:
      path = Path()
      path.route = shortest_path.route[:shortest_path.route.index(vertex)+1]
    #   print(f"path route: {path.route}")

      flag = False
    #   print(f"neighbor: {neighbor}")

      if not path.contains(neighbor):
        if shortest_path.contains(neighbor):
          if shortest_path.route.index(vertex)+1 != shortest_path.route.index(neighbor) :
            path.route.append(neighbor)
            flag = True
            # print(f"{neighbor} route a eklendi")
          else:
            continue

        else:
          path.route.append(neighbor)
          flag = True
        #   print(f"{neighbor} route a eklendi")
      
      if flag:
        for i in range(len(path.route)-1):
          u, v = path.route[i], path.route[i+1]
          path.edges[(u,v)] = graph[u][v]['weight']
          path.length += graph[u][v]['weight']
        path.cls = (1, vertex)
        path.lb = path.LB1(vertex, distances)
        heapq.heappush(global_PQ, path)

    # for a in global_PQ:
    #   print(a)

  while len(result_set) < k and global_PQ:
    # current_path = heapq.heappop(global_PQ)

    # if current_path.tail() == dest:
    #   if current_path.Sim(threshold, result_set):
    #     result_set.append(current_path)

    # else:
    #   extended_paths = ExtendPath(current_path)

    # for path in extended_paths:
    #   heapq.heappush(global_PQ, path)

    new_path = FindNextPath(graph, global_PQ, threshold)
    if new_path.Sim(threshold, result_set):
      result_set.append(new_path)

  return result_set
 

In [ ]:
FindKSPD(G, 1, 4, 3, 0.5)